# LLM test (depreciated no longer used)

In [ ]:
# Install with: pip install llama-cpp-python
from llama_cpp import Llama


# Model path - change this to where you download the model
model_path = "../Models/tinyllama-1.1b-chat-v1.0.Q8_0.gguf"

# Load the model
llm = Llama(
    model_path=model_path,
    n_ctx=512,         # Small context window for faster inference (context win = how much text the model can consider at once) (inc = better at long prompts but more RAM dec = faster less ram but worse at long prompts)
    n_threads=4,       # Adjust based on your CPU cores (num of cpu cors to use for parallel processing) (inc = more cpu usage dec = less cpu usage)
    n_batch=32,        # Lower batch size for faster inference (num of tokens to process at once) (inc = more tokens at once hencse better throughput dec = less tokens at once)
    f16_kv=True,          # Use half-precision for key/value cache
    use_mlock=True,       # Keep model in RAM
    verbose=False,
    n_gpu_layers=-1 
)

# Test the model
output = llm(
    "Once upon a time,",
    max_tokens=128,     # Reduced for faster response
    echo=True
)
print(output["choices"][0]["text"])